In [21]:
import os
from pathlib import Path

In [29]:
os.chdir(Path("C:/Users/Hp/Videos/classification implementation - machine learning with MLFlow"))

In [30]:
%pwd

'C:\\Users\\Hp\\Videos\\classification implementation - machine learning with MLFlow'

In [31]:
import pandas as pd

In [32]:
data = pd.read_csv('artifacts/data_ingestion/classification.csv')
data.head()

,label,account_type,credit_duration(month),payment_status_of_previous_loan,loan_purpose,credit_amount,savings_type,duration_of_current_employment,installment_percent,marital_status,...,duration_in_current_address,most_valuable_asset,age,concurrent_credits,type_of_apartment,no_of_credits_at_the_bank,occupation,no_of_dependents,telephone,foreign_worker
0,0,savings,18,paid,travel and vacations,1049,high-yield savings accounts,2,4,divorced,...,4,intellectual property,21,3,two or multi-bedroom apartment,1,service and sales,1,1,no
1,0,savings,9,paid,purchase a vehicle,2799,high-yield savings accounts,3,2,married,...,2,human capital,36,3,two or multi-bedroom apartment,2,service and sales,2,1,no
2,0,current,12,opted to pay,student loan,841,individual retirement accounts,4,2,divorced,...,4,human capital,23,3,two or multi-bedroom apartment,1,skilled trades and technical,1,1,no
3,0,savings,12,paid,purchase a vehicle,2122,high-yield savings accounts,3,3,married,...,2,human capital,39,3,two or multi-bedroom apartment,2,skilled trades and technical,2,1,yes
4,0,savings,12,paid,purchase a vehicle,2171,high-yield savings accounts,3,4,married,...,4,intellectual property,38,1,one-bedroom apartment,2,skilled trades and technical,1,1,yes


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   label                            1000 non-null   int64 
 1   account_type                     1000 non-null   object
 2   credit_duration(month)           1000 non-null   int64 
 3   payment_status_of_previous_loan  1000 non-null   object
 4   loan_purpose                     1000 non-null   object
 5   credit_amount                    1000 non-null   int64 
 6   savings_type                     1000 non-null   object
 7   duration_of_current_employment   1000 non-null   int64 
 8   installment_percent              1000 non-null   int64 
 9   marital_status                   1000 non-null   object
 10  guarantors                       1000 non-null   int64 
 11  duration_in_current_address      1000 non-null   int64 
 12  most_valuable_asset              10

In [33]:
# Schema validation class
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [34]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [35]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            unzip_data_dir = config.unzip_data_dir,
            STATUS_FILE= config.STATUS_FILE,
            all_schema = schema,
        )

        return data_validation_config

In [36]:
import os
from mlProject import logger

In [40]:
class DataValidation:
    def __init__(self, config) -> DataValidationConfig:
        self.config = config
    
    def validate_all_columns(self) -> bool:
        try:
            validation_status = None
            
            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"validation status: {validation_status}")
            return validation_status
        
        except Exception as e:
            raise e


In [45]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise

[2023-09-21 21:20:33,262: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-21 21:20:33,271: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-21 21:20:33,286: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-21 21:20:33,288: INFO: common: created directory at: artifacts]
[2023-09-21 21:20:33,290: INFO: common: created directory at: artifacts/data_validation]
